# Import des modules nécessaires à l'ensemble

In [1]:
import pandas as pd

from sqlalchemy import create_engine

# 1. Lecture de données et réflexion sur le modèle

On importe des donnnées dans `pandas`.

In [2]:
# Je force le type à str afin qu'il ne convertisse rien en int ou float
df = pd.read_csv('sirene-dummy.csv', dtype=str)

df

,SIREN,NIC,NOM ENTREPRISE,NOM ETABLISSEMENT,VILLE
0,123456789,00001,FNAC,FNAC ANGERS,ANGERS
1,123456789,00002,FNAC,FNAC ROUEN,ROUEN
2,123456789,00003,FNAC,FNAC PARIS,PARIS
3,987654321,00001,COTO,COTO,ANGERS


Imaginons que ce soit le "fichier du jour" de la base SIRENE, et qu'on veuille le conserver dans une base de données locale (l'historiser).

On peut simplement faire :

In [3]:
# On crée une base sqlite bidon, dans la RAM du PC, juste pour l'exercice
engine = create_engine('sqlite://')
connection = engine.connect()

In [4]:
# On écrit la table (il la crée, du coup).
df.to_sql('stock-sirene', connection, if_exists='replace', index=False)

In [5]:
# Vérifions
pd.read_sql_table('stock-sirene', connection)

,SIREN,NIC,NOM ENTREPRISE,NOM ETABLISSEMENT,VILLE
0,123456789,00001,FNAC,FNAC ANGERS,ANGERS
1,123456789,00002,FNAC,FNAC ROUEN,ROUEN
2,123456789,00003,FNAC,FNAC PARIS,PARIS
3,987654321,00001,COTO,COTO,ANGERS


In [6]:
connection.close()

Ce n'est cependant **pas très satisfaisant** :
- Certaines données sont inutilement répétées (le nom de l'entreprise, de la ville).
- On ne fait pas l'effort de donner des informations sur la structure des données.

# 2. Un nouvel espoir : le modèle de données

En vrai, on a (au moins) **trois types d'objets en présence** :
- une entreprise,
- un établissement,
- une ville.

Décrivons-les.

## Entreprises

In [7]:
# On a besoin du SIREN et du nom de l'entreprise
dft = df[['SIREN', 'NOM ENTREPRISE']].drop_duplicates()
dft = dft.rename(columns={
    'SIREN': 'id',
    'NOM ENTREPRISE': 'name'
})

dft

,id,name
0,123456789,FNAC
3,987654321,COTO


In [8]:
# Et on envoie dans SQL
with engine.connect() as connection:
    dft.to_sql('entreprises', connection, if_exists='replace')

## Etablissements

In [9]:
# On a besoin du SIRET, du nom d'établissement et de la ville
dft = df[['SIREN', 'NIC', 'NOM ETABLISSEMENT', 'VILLE']]
dft['SIRET'] = dft['SIREN'] + dft['NIC']
dft = dft.drop(columns=['NIC'])

dft = dft.rename(columns={
    'NOM ETABLISSEMENT': 'name',
    'VILLE': 'cityname',
    'SIRET': 'id',
    'SIREN': 'companyid'
})

dft

,companyid,name,cityname,id
0,123456789,FNAC ANGERS,ANGERS,12345678900001
1,123456789,FNAC ROUEN,ROUEN,12345678900002
2,123456789,FNAC PARIS,PARIS,12345678900003
3,987654321,COTO,ANGERS,98765432100001


In [10]:
# Et on envoie dans SQL
with engine.connect() as connection:
    dft.to_sql('etablissements', connection, if_exists='replace')

**Remarque :** On pourrait également faire une table pour les villes, et du coup les tables entreprises et établissements ne contiendrait que par exemple le code INSEE de la ville.

# 3. Alors pourquoi un ORM ?

On a vu qu'il était très facile et efficace d'utiliser directement les outils de base de `SQLAlchemy` pour exporter dans une base SQL.

Du côté requête, l'ORM va cependant être très pratique.

Par ailleurs, il nous oblige à structurer encore plus notre modèle.

## Besoin de 2-3 choses en plus

In [11]:
from sqlalchemy import Column, create_engine, ForeignKey, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

In [12]:
Base = declarative_base()

In [13]:
class Company(Base):
    # le nom de la table dans la base
    __tablename__ = "entreprises"
    
    # les définitions des colonnes (avec plus de précisions)
    id = Column(String(9), primary_key=True) # SIREN
    name = Column(String)
    
    # relation : ça ne va être utile qu'en Python, ça ne change rien à la table.
    branches = relationship("Branch", back_populates="company")
    
    # et ça c'est juste pour afficher proprement les objets "Company"
    def __repr__(self):
        return "<Company(id={}, name={})>".format(self.id, self.name)

In [14]:
class Branch(Base):
    # le nom de la table dans la base
    __tablename__ = "etablissements"
    
    # les définitions des colonnes (avec plus de précisions)
    id = Column(String(14), primary_key=True) # SIRET
    companyid = Column(String(9), ForeignKey('entreprises.id')) # SIREN
    name = Column(String)
    cityname = Column(String)
    
    # relation : ça ne va être utile qu'en Python, ça ne change rien à la table.
    company = relationship("Company", back_populates="branches")
    
    # et ça c'est juste pour afficher proprement les objets "Branch"
    def __repr__(self):
        return "<Branch(id={}, name={})>".format(self.id, self.name)

On exécute cela une fois pour créer le schéma dans la base et le lier avec le modèle Python.

In [15]:
Base.metadata.create_all(bind=engine)

## Maintenant on peut requêter

In [16]:
Session = sessionmaker(bind=engine)
session = Session()

In [17]:
session.query(Branch).all()

[<Branch(id=12345678900001, name=FNAC ANGERS)>,
 <Branch(id=12345678900002, name=FNAC ROUEN)>,
 <Branch(id=12345678900003, name=FNAC PARIS)>,
 <Branch(id=98765432100001, name=COTO)>]

Remarque que ce que l'on obtient en sortie est un vrai objet Python, pas juste une liste de lignes :

In [18]:
b = session.query(Branch).first()

print(type(b))

<class '__main__.Branch'>


In [19]:
session.close()

## Et faire des requêtes plus complexes/sympa

Pour en voir plus : http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#querying

In [20]:
session = Session()

In [21]:
c = session.query(Company) \
           .filter_by(name='FNAC') \
           .first()

c

<Company(id=123456789, name=FNAC)>

Et à quoi servaient les attributs créés avec la fonction `relationship` ?

In [22]:
print(c.branches)

[<Branch(id=12345678900001, name=FNAC ANGERS)>, <Branch(id=12345678900002, name=FNAC ROUEN)>, <Branch(id=12345678900003, name=FNAC PARIS)>]


On peut alors facilement boucler dessus, etc. La syntaxe est assez agréable.

In [23]:
for branch in c.branches:
    print(branch.name)

FNAC ANGERS
FNAC ROUEN
FNAC PARIS


Pour les `join` etc., voir ici : http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#querying-with-joins

In [24]:
session.close()

## Export dans un DataFrame

In [25]:
session = Session()

In [30]:
pd.read_sql_query(session.query(Branch).statement, session.bind)

<Company(id=123456789, name=FNAC)>

In [29]:
session.close()

# 4. Y a-t-il une morale à l'histoire ?

Personnellement, j'aime beaucoup l'ORM car il offre une *vraie structure* à notre base SQL. De plus, il instancie un parallèle que je trouve élégant entre une table en SQL et un objet en Python. Enfin, il rend les INSERT et les SELECT bien plus faciles à comprendre : *la syntaxe donne vraiment le sentiment de manipuler des entités/objets*.

Cependant, pour ce qui est de remplir *massivement* une base de données, il peut être assez inefficace (voir [ici](http://docs.sqlalchemy.org/en/latest/faq/performance.html#i-m-inserting-400-000-rows-with-the-orm-and-it-s-really-slow)) et, tant que possible, je lui préférerai `pandas` et la méthode `to_sql()`.

A l'avenir, je préconiserais donc d'utiliser `to_sql` pour remplir la base, et `SQLAlchemy` ORM pour la requêter.